<center><b>Funciones para Endspoints (API)<b><center>

In [92]:
# Importacion de librerias
import pandas as pd
import ast
import pdb

In [93]:
# Carga de datasets
df_items = pd.read_parquet('..\\datasets\\New_datasets\\user_items.parquet')
df_reviews = pd.read_parquet('..\\datasets\\New_datasets\\user_reviews_sentiments.parquet')
df_steams = pd.read_csv('..\\datasets\\New_datasets\\user_steams.csv')

C:\Users\DELL\AppData\Local\Temp\ipykernel_14948\1214550425.py:4: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_steams = pd.read_csv('..\\datasets\\New_datasets\\user_steams.csv')


In [94]:
df_items['steam_id'] = df_items['steam_id'].apply(float)
df_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5153209 entries, 0 to 5153208
Data columns (total 6 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   items_count       int64  
 2   steam_id          float64
 3   item_id           object 
 4   item_name         object 
 5   playtime_forever  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 235.9+ MB


<b>PlayTimeGenre:</b>       

Debe devolver año con mas horas jugadas para dicho género.

In [95]:
# Se eliminan los datos nulos de las columnas a consultar
df_steams_genre = df_steams.dropna(subset='genres')
df_steams_genre.reset_index(inplace=True,drop=True)
df_steams_genre = df_steams_genre.dropna(subset='release_year')
df_steams_genre.reset_index(inplace=True,drop=True)
df_steams_genre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28660 entries, 0 to 28659
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     23751 non-null  object 
 1   genres        28660 non-null  object 
 2   app_name      28659 non-null  object 
 3   title         28659 non-null  object 
 4   url           28660 non-null  object 
 5   release_date  28660 non-null  object 
 6   tags          28636 non-null  object 
 7   reviews_url   28659 non-null  object 
 8   specs         28374 non-null  object 
 9   price         27615 non-null  object 
 10  early_access  28660 non-null  object 
 11  id            28659 non-null  float64
 12  developer     28490 non-null  object 
 13  release_year  28660 non-null  float64
dtypes: float64(2), object(12)
memory usage: 3.1+ MB


In [96]:
# Desanidado de columna genres de dataset df_steams_genre
datos = []
def genre_reader(df_genre):
    try:
        for index,line in enumerate(df_genre['genres']):
            dato_genres = ast.literal_eval(line) 
            #pdb.set_trace()
            for genre in dato_genres:
                #pdb.set_trace()
                datos.append([genre,df_genre['release_year'][index],df_genre['app_name'][index],df_genre['title'][index]])
    except:
        pdb.set_trace()

    return pd.DataFrame(datos,columns=['genres','release_year','app_name','title'])

In [97]:
# Se observa que se presentan columna duplicadas
df_PlayTimeGenre = genre_reader(df_steams_genre)
# Se trabaja con una de las columnas nombre y se rescata solo las columnas de interes
df_PlayTimeGenre = df_PlayTimeGenre.merge(df_items,left_on='title', right_on='item_name')
df_PlayTimeGenre = df_PlayTimeGenre[['genres','release_year','playtime_forever']]

# Se agrupa 'genres','release_year' y se suma la columna 'playtime_forever' segun lo solicitado. 
# Se utiliza la funcion transform para especificar el contenido de la nueva columna
df_PlayTimeGenre['Sum_playtime_forever'] = df_PlayTimeGenre.groupby(['genres','release_year'])['playtime_forever'].transform('sum')

# Se elimina la columna 'playtime_forever' que ya se utilizo para la suma y se quita duplicados generados en group by
df_PlayTimeGenre.drop(columns='playtime_forever',inplace=True)
df_PlayTimeGenre.drop_duplicates(subset=['genres','release_year'],inplace=True)
df_PlayTimeGenre.reset_index(drop=True,inplace=True)
df_PlayTimeGenre.info()
df_PlayTimeGenre.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 341 entries, 0 to 340
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   genres                341 non-null    object 
 1   release_year          341 non-null    float64
 2   Sum_playtime_forever  341 non-null    int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 8.1+ KB


,genres,release_year,Sum_playtime_forever
0,Action,1997.0,271013
1,Indie,1997.0,64814
2,Racing,1997.0,9319


In [98]:
# Funcion de consulta por genero el año de mayor cantidad de horas acumuladas
def PlayTimeGenre(genre):
    anio = df_PlayTimeGenre['release_year'][df_PlayTimeGenre['Sum_playtime_forever'][df_PlayTimeGenre['genres'] == genre].idxmax()]
    return str('Año de lanzamiento con más horas jugadas para Género ' + genre + ': ' + str(anio))

In [99]:
genre = 'Action'
anio = 2012
'Año de lanzamiento con más horas jugadas para Género ' + genre + ': ' + str(anio)

'Año de lanzamiento con más horas jugadas para Género Action: 2012'

In [100]:
df_PlayTimeGenre[df_PlayTimeGenre['genres'] == 'Action'][df_PlayTimeGenre['genres'] == 'Action']

C:\Users\DELL\AppData\Local\Temp\ipykernel_14948\2605478342.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_PlayTimeGenre[df_PlayTimeGenre['genres'] == 'Action'][df_PlayTimeGenre['genres'] == 'Action']


,genres,release_year,Sum_playtime_forever
0,Action,1997.0,271013
3,Action,1998.0,2809535
5,Action,2005.0,6015794
13,Action,2000.0,18553775
17,Action,2007.0,11309542
19,Action,1996.0,51726
20,Action,1994.0,40603
21,Action,2001.0,1255120
22,Action,2003.0,9906989
24,Action,1999.0,2721215


In [101]:
# Prueba
PlayTimeGenre('Action')

'Año de lanzamiento con más horas jugadas para Género Action: 2012.0'

<b>UserForGenre:</b>       

Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año. Es similar la funcion PlayTimeGenre, pero se agrega usuario y lista de juegos con horas jugadas

In [102]:
# Consulta de generos desanidados y merge con dataframe items
df_UserForGenre = genre_reader(df_steams_genre)
df_UserForGenre = df_UserForGenre.merge(df_items,left_on='title', right_on='item_name')
df_UserForGenre = df_UserForGenre[['genres','user_id','release_year','playtime_forever']]


In [103]:
# Agrupado segun lo requerido
df_UserForGenre['Sum_playtime_year'] = df_UserForGenre.groupby(['genres','user_id','release_year'])['playtime_forever'].transform('sum')
df_UserForGenre.drop(columns='playtime_forever',inplace=True)
df_UserForGenre.drop_duplicates(subset=['genres','user_id','release_year'],inplace=True)
df_UserForGenre.reset_index(drop=True,inplace=True)
df_UserForGenre_group = df_UserForGenre.copy()

In [104]:
df_UserForGenre_group.head(3)

,genres,user_id,release_year,Sum_playtime_year
0,Action,UTNerd24,1997.0,2386
1,Action,I_DID_911_JUST_SAYING,1997.0,0
2,Action,76561197962104795,1997.0,0


In [105]:
df_genres = df_UserForGenre_group['genres']
df_genres.drop_duplicates(inplace=True)
df_genres.reset_index(drop=True,inplace=True)
df_genres

0                        Action
1                         Indie
2                        Racing
3                      Strategy
4                    Simulation
5                        Casual
6                     Adventure
7                           RPG
8                        Sports
9                  Free to Play
10        Massively Multiplayer
11                 Early Access
12                    Education
13            Software Training
14                    Utilities
15               Web Publishing
16    Design &amp; Illustration
17     Animation &amp; Modeling
18                Photo Editing
19             Audio Production
20             Video Production
Name: genres, dtype: object

In [106]:
# Funcion de agrupamiento y consulta
def UserForGenre(genre):
    # Captando el maximo de tiempo
    user = df_UserForGenre_group['user_id'][df_UserForGenre_group['Sum_playtime_year'][df_UserForGenre_group['genres'] == genre].idxmax()]
    lista = df_UserForGenre_group[df_UserForGenre_group['genres'] == genre][df_UserForGenre_group['user_id'] == user]
    lista = lista[['release_year','Sum_playtime_year']]
    lista.sort_values(by = ['release_year'], inplace = True)
    lista.reset_index(drop = True,inplace = True)
    #pdb.set_trace()
    lista_str = ''
    for index,line in enumerate(lista['release_year']):
        # El tiempo estaba en minutos, se convierten a horas y redondea 2 decimales por presentacion
        horas = str(round(lista['Sum_playtime_year'][index]/60,2))
        lista_str = lista_str + str(' Año:' +str(int(line)) + ' Horas: '+ horas)
    return {'user' : user, 'lista' : lista_str}

In [107]:
data = UserForGenre('Action')

C:\Users\DELL\AppData\Local\Temp\ipykernel_14948\3567891882.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  lista = df_UserForGenre_group[df_UserForGenre_group['genres'] == genre][df_UserForGenre_group['user_id'] == user]


In [108]:
data

{'user': 'Evilutional',
 'lista': ' Año:2003 Horas: 0.0 Año:2006 Horas: 0.0 Año:2009 Horas: 116.43 Año:2010 Horas: 5.07 Año:2011 Horas: 54.83 Año:2012 Horas: 22609.63 Año:2013 Horas: 2324.2 Año:2014 Horas: 68.37 Año:2015 Horas: 3.73 Año:2017 Horas: 363.13'}

In [109]:

def UserForGenre_total():
    return {genre: UserForGenre(genre) for genre in df_genres}
df_UserForGenre_Total = UserForGenre_total()

C:\Users\DELL\AppData\Local\Temp\ipykernel_14948\3567891882.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  lista = df_UserForGenre_group[df_UserForGenre_group['genres'] == genre][df_UserForGenre_group['user_id'] == user]
C:\Users\DELL\AppData\Local\Temp\ipykernel_14948\3567891882.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  lista = df_UserForGenre_group[df_UserForGenre_group['genres'] == genre][df_UserForGenre_group['user_id'] == user]
C:\Users\DELL\AppData\Local\Temp\ipykernel_14948\3567891882.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  lista = df_UserForGenre_group[df_UserForGenre_group['genres'] == genre][df_UserForGenre_group['user_id'] == user]
C:\Users\DELL\AppData\Local\Temp\ipykernel_14948\3567891882.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  lista = df_UserForGenre_group[df_UserForGenre_group['genres'] == genre][df_UserFor

In [110]:
df_UserForGenre_Total.get('Indie')

{'user': 'REBAS_AS_F-T',
 'lista': ' Año:1999 Horas: 0.0 Año:2001 Horas: 0.37 Año:2003 Horas: 62.1 Año:2005 Horas: 0.0 Año:2006 Horas: 55.77 Año:2007 Horas: 0.0 Año:2008 Horas: 45.53 Año:2009 Horas: 901.0 Año:2010 Horas: 716.2 Año:2011 Horas: 2832.9 Año:2012 Horas: 4031.33 Año:2013 Horas: 5435.53 Año:2014 Horas: 9331.0 Año:2015 Horas: 23997.83 Año:2016 Horas: 25836.23 Año:2017 Horas: 1072.03'}

In [111]:
df_UserForGenre_Total = pd.DataFrame(df_UserForGenre_Total)
df_UserForGenre_Total

,Action,Indie,Racing,Strategy,Simulation,Casual,Adventure,RPG,Sports,Free to Play,...,Early Access,Education,Software Training,Utilities,Web Publishing,Design &amp; Illustration,Animation &amp; Modeling,Photo Editing,Audio Production,Video Production
user,Evilutional,REBAS_AS_F-T,DownSyndromeKid,shinomegami,clawbot44,idonothack,REBAS_AS_F-T,Evilutional,1silentsniper1,idonothack,...,76561197978756659,76561198059330972,Lickidactyl,76561198073642113,76561198035718256,76561198035718256,76561198059330972,Colidos,Lickidactyl,Lickidactyl
lista,Año:2003 Horas: 0.0 Año:2006 Horas: 0.0 Año:2...,Año:1999 Horas: 0.0 Año:2001 Horas: 0.37 Año:...,Año:1994 Horas: 0.0 Año:1995 Horas: 10.6 Año:...,Año:1993 Horas: 733.37 Año:1997 Horas: 0.0 Añ...,Año:2006 Horas: 24105.4 Año:2007 Horas: 35.47...,Año:1994 Horas: 0.0 Año:2002 Horas: 0.0 Año:2...,Año:2002 Horas: 0.0 Año:2003 Horas: 62.1 Año:...,Año:2011 Horas: 53.8 Año:2012 Horas: 21319.17...,Año:2009 Horas: 9.13 Año:2013 Horas: 83.73 Añ...,Año:2003 Horas: 0.0 Año:2006 Horas: 0.6 Año:2...,...,Año:2013 Horas: 10559.53 Año:2015 Horas: 6.1,Año:2013 Horas: 2180.9,Año:2014 Horas: 3663.87,Año:2014 Horas: 6921.7,Año:2012 Horas: 3418.47 Año:2015 Horas: 414.27,Año:2012 Horas: 3418.47 Año:2015 Horas: 414.27,Año:2013 Horas: 2203.47,Año:2016 Horas: 250.7,Año:2014 Horas: 3663.87,Año:2014 Horas: 3663.87


In [112]:
df_UserForGenre_Total['Action']['user']

'Evilutional'

<b>UsersRecommend:</b>       

Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)

In [113]:
df_reviews_recomend = df_reviews.dropna(subset='recommend')
df_reviews_recomend.reset_index(drop=True,inplace=True)
df_reviews_recomend.info()
df_reviews_recomend.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49186 entries, 0 to 49185
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             49186 non-null  object
 1   item_id             49186 non-null  object
 2   recommend           49186 non-null  bool  
 3   review              49186 non-null  object
 4   year_posted         49186 non-null  int64 
 5   sentiment_analysis  49186 non-null  int64 
dtypes: bool(1), int64(2), object(3)
memory usage: 1.9+ MB


,user_id,item_id,recommend,review,year_posted,sentiment_analysis
0,76561197970982479,1250,True,Simple yet with great replayability. In my opi...,2011,1
1,76561197970982479,22200,True,It's unique and worth a playthrough.,2011,1
2,76561197970982479,43110,True,Great atmosphere. The gunplay can be a bit chu...,2011,1
3,js41637,251610,True,I know what you think when you see this title ...,2014,1
4,js41637,227300,True,For a simple (it's actually not all that simpl...,2013,1


In [114]:
df_UsersRecommend = df_reviews_recomend[['item_id','recommend','year_posted','sentiment_analysis']].copy()
df_UsersRecommend = df_UsersRecommend[df_UsersRecommend['recommend']]

# Separando comentarios positivos y neutrales
df_UsersRecommend = df_UsersRecommend[df_UsersRecommend['sentiment_analysis'] >= 1]

df_UsersRecommend['Recomends_times'] = df_UsersRecommend.groupby(['item_id','year_posted'])['item_id'].transform('count')
df_UsersRecommend.drop_duplicates(subset=['item_id','year_posted','Recomends_times'],inplace=True)
df_UsersRecommend.reset_index(drop=True,inplace=True)
df_UsersRecommend.sort_values(by='Recomends_times',ascending=False,inplace=True)

df_UsersRecommend.head()


,item_id,recommend,year_posted,sentiment_analysis,Recomends_times
26,440,True,2014,1,1536
38,730,True,2015,1,1502
19,730,True,2014,1,1055
71,440,True,2013,1,788
32,4000,True,2014,1,754


In [115]:
# Funcion de consulta
def UsersRecommend(anio):
    top_recomend = df_UsersRecommend[df_UsersRecommend['year_posted'] == anio]
    top_recomend.reset_index(drop = True,inplace = True)
    return str('Items_id Puesto 1: '+ top_recomend['item_id'][0] + ' Puesto 2: '+ top_recomend['item_id'][1] + ' Puesto 3: '+ top_recomend['item_id'][2])

In [116]:
top_recomend = UsersRecommend(2014)

In [117]:
top_recomend

'Items_id Puesto 1: 440 Puesto 2: 730 Puesto 3: 4000'

<b>UsersNotRecommend:</b>       

Devuelve el top 3 de juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)

In [118]:
df_UsersNotRecommend = df_reviews_recomend[['item_id','recommend','year_posted','sentiment_analysis']].copy()
df_UsersNotRecommend = df_UsersNotRecommend[df_UsersNotRecommend['recommend'] == 0]

# Separando comentarios negativos
df_UsersNotRecommend = df_UsersNotRecommend[df_UsersNotRecommend['sentiment_analysis'] == 0]

df_UsersNotRecommend['Recomends_times'] = df_UsersNotRecommend.groupby(['item_id','year_posted'])['item_id'].transform('count')
df_UsersNotRecommend.drop_duplicates(subset=['item_id','year_posted','Recomends_times'],inplace=True)
df_UsersNotRecommend.reset_index(drop=True,inplace=True)
df_UsersNotRecommend.sort_values(by='Recomends_times',ascending=True,inplace=True)
df_UsersNotRecommend

,item_id,recommend,year_posted,sentiment_analysis,Recomends_times
234,305920,False,2015,0,1
200,208730,False,2015,0,1
123,2870,False,2014,0,1
124,331440,False,2015,0,1
125,42680,False,2014,0,1
...,...,...,...,...,...
9,51100,False,2014,0,6
46,304930,False,2015,0,7
36,218620,False,2015,0,8
75,730,False,2015,0,9


In [119]:
# Funcion de consulta
def UsersNotRecommend(anio):
    top_not_recomend = df_UsersNotRecommend[df_UsersNotRecommend['year_posted'] == anio]
    top_not_recomend.reset_index(drop = True,inplace = True)
    return str('Items_id Puesto 1: '+ top_not_recomend['item_id'][0] + ' Puesto 2: '+ top_not_recomend['item_id'][1] + ' Puesto 3: '+ top_not_recomend['item_id'][2])

In [120]:
UsersNotRecommend(2014)

'Items_id Puesto 1: 2870 Puesto 2: 42680 Puesto 3: 332070'

<b>sentiment_analysis:</b>       

Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.

In [121]:
df_reviews.head()

,user_id,item_id,recommend,review,year_posted,sentiment_analysis
0,76561197970982479,1250,True,Simple yet with great replayability. In my opi...,2011,1
1,76561197970982479,22200,True,It's unique and worth a playthrough.,2011,1
2,76561197970982479,43110,True,Great atmosphere. The gunplay can be a bit chu...,2011,1
3,js41637,251610,True,I know what you think when you see this title ...,2014,1
4,js41637,227300,True,For a simple (it's actually not all that simpl...,2013,1


In [122]:
df_SentimentRecommend = df_reviews[['item_id','year_posted','sentiment_analysis']].copy()

# Agrupando por 'year_posted','sentiment_analysis' y contando la cantidad de datos
df_SentimentRecommend['Recomends_times'] = df_SentimentRecommend.groupby(['item_id','year_posted','sentiment_analysis'])['year_posted'].transform('count')
df_SentimentRecommend.drop_duplicates(subset=['item_id','year_posted','sentiment_analysis'],inplace=True)
df_SentimentRecommend.sort_values(by='sentiment_analysis',ascending=True,inplace=True)
df_SentimentRecommend.reset_index(drop=True,inplace=True)


In [123]:
# Cambiando contenido de sentiment_analysis
df_SentimentRecommend['sentiment_analysis'].replace(0,'Negative',inplace=True)
df_SentimentRecommend['sentiment_analysis'].replace(1,'Neutral',inplace=True)
df_SentimentRecommend['sentiment_analysis'].replace(2,'Positive',inplace=True)

df_SentimentRecommend

,item_id,year_posted,sentiment_analysis,Recomends_times
0,12210,2014,Negative,3
1,630,2013,Negative,1
2,33950,2014,Negative,1
3,204080,2014,Negative,1
4,322920,2015,Negative,1
...,...,...,...,...
6875,221430,2013,Positive,1
6876,10,2012,Positive,2
6877,49540,2013,Positive,1
6878,42719,2014,Positive,1


In [124]:
df_SentimentRecommend.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6880 entries, 0 to 6879
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   item_id             6880 non-null   object
 1   year_posted         6880 non-null   int64 
 2   sentiment_analysis  6880 non-null   object
 3   Recomends_times     6880 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 215.1+ KB


In [125]:
anio = 2014
analisis_sentimiento = df_SentimentRecommend[df_SentimentRecommend['year_posted'] == anio]
analisis_sentimiento.count()

item_id               2374
year_posted           2374
sentiment_analysis    2374
Recomends_times       2374
dtype: int64

In [126]:
def sentiment_analysis(anio):
    analisis_sentimiento = df_SentimentRecommend[df_SentimentRecommend['year_posted'] == anio]

    return print('Negative: ', analisis_sentimiento['sentiment_analysis'][analisis_sentimiento['sentiment_analysis'] == 'Negative'].count(),
                 'Neutral: ', analisis_sentimiento['sentiment_analysis'][analisis_sentimiento['sentiment_analysis'] == 'Neutral'].count(),
                 'Positive: ', analisis_sentimiento['sentiment_analysis'][analisis_sentimiento['sentiment_analysis'] == 'Positive'].count())

In [127]:
sentiment_analysis(2014)

Negative:  223 Neutral:  1512 Positive:  639


<b>Guardar dataframes para Funciones</b>

In [128]:
df_PlayTimeGenre.to_parquet('..\\Funciones\\datasets\\PlayTimeGenre.parquet',index=False)
df_UserForGenre_Total.to_parquet('..\\Funciones\\datasets\\UserForGenre_total.parquet',index=False)
df_UserForGenre.to_parquet('..\\Funciones\\datasets\\UserForGenre.parquet',index=False)
df_UsersRecommend.to_parquet('..\\Funciones\\datasets\\UsersRecommend.parquet',index=False)
df_UsersNotRecommend.to_parquet('..\\Funciones\\datasets\\UsersNotRecommend.parquet',index=False)
df_SentimentRecommend.to_parquet('..\\Funciones\\datasets\\SentimentRecommend.parquet',index=False)